## Romberg integration
Reference: Newman, Section 5.4 Romberg integration

In [11]:
import numpy as np

In [12]:
initial_bins = 10 #initial number of bins for the trapezoidal rule
romberg_steps = 3 #romberg_steps=1 is the standard trapezoidal rule

#bounds of integration
lower_bound = 0
upper_bound = 1 #np.pi

In [13]:
def integrand(x): #integrand function
    return x**4

In [14]:
def integrate_trapezoidal(f, lower_bound, upper_bound, bins):
    points = bins + 1
    bin_width = (upper_bound - lower_bound)/bins
    x = np.linspace(lower_bound, upper_bound, points)
    integral = bin_width * (np.sum(f(x[1:-1])) +  
                0.5 * (f(lower_bound) + f(upper_bound)))
    return integral

Apply trapezoidal rule for the initial number of bins to get $I_1$<br>
$R_{1,1} = I_1$ <br> 
When we double the number of bins, we get $I_2$. Double the number of bins again and we get $I_3$, and so on. Denote $R_{i,1}$ to be this trapezoidal rule output $I_i$, i.e. <br> 
$R_{i,1} = I_i$ <br> 
The correction factor that improves the order of error of the estimate from successive splitting of bins comes from: <br>
$R_{i,m+1} = R_{i,m} + \frac{1}{4^m-1}(R_{i,m} − R_{i−1,m})$<br>
Because of this recursion relation, the best estimate we'll get would be $R_{n,n}$, accurate to the order of $h_n^{2n}$ if we do $n-1$ successive doubling of the number of bins. However, we do not have error estimate for this term -- the one we have is for $R_{n,n-1}$, which we used to obtain $R_{n,n}$.

In [15]:
romberg_term = np.zeros([romberg_steps+1, romberg_steps+1])
bins = initial_bins
for i in range(1,romberg_steps+1):
    romberg_term[i, 1] = integrate_trapezoidal(integrand, lower_bound, upper_bound, bins=bins)
    bins = 2*bins
    
    for m in range(1,i):
        coeff = 1./(4**m-1)
        romberg_term[i,m+1] = romberg_term[i,m] + coeff*(romberg_term[i,m] - romberg_term[i-1,m])
print(romberg_term[1:,1:])
print("Best estimate: ", romberg_term[romberg_steps, romberg_steps])
print("Approximation error should be better than: ", np.abs(romberg_term[romberg_steps, romberg_steps] -romberg_term[romberg_steps, romberg_steps-1]))
print("Upper bound relative error due to floating point arithmetic: ", np.finfo(float).eps)

[[0.20333    0.         0.        ]
 [0.20083313 0.20000083 0.        ]
 [0.20020832 0.20000005 0.2       ]]
Best estimate:  0.20000000000000007
Approximation error should be better than:  5.2083333340613436e-08
Upper bound relative error due to floating point arithmetic:  2.220446049250313e-16


Notes: <br>
1. We could have adjusted the indexing by creating another function on top of the direct call to the numpy array. We didn't do that here because the waste in memory is insignificant, this will not always be the case.
2. Expressing the correct number of significant figures is left to the student.
3. In the call: integrate_trapezoidal(integrand, lower_bound, upper_bound, bins=bins), bins=bins is not a weird piece of code -- it is assigning the value of bins inside the integrate_trapezoidal function with the value of bins assigned in the loop. It could have very been written up as bins=kahon, where kahon = 2 * kahon, and kahon is set to initial_bins before enter